In [2]:
def probAttr(data, attr, val):
    Total = data.shape[0]
    cnt = len(data[data[attr] == val])
    return cnt, cnt / Total

In [3]:
def train(data, Attr, conceptVals, concept):
    conceptProbs = {}
    countConcept = {}
    for cVal in conceptVals:
        countConcept[cVal], conceptProbs[cVal] = probAttr(data, concept, cVal)
    AttrConcept = {}
    probability_list = {}
    for att in Attr:
        probability_list[att] = {}
        AttrConcept[att] = {}
        for val in Attr[att]:
            AttrConcept[att][val] = {}
            a, probability_list[att][val] = probAttr(data, att, val)
            dataTemp = data[data[att] == val]
            for cVal in conceptVals:
                AttrConcept[att][val][cVal] = len(dataTemp[dataTemp[concept] ==cVal]) / countConcept[cVal]
    print(f"P(A) : {conceptProbs}\n")
    print(f"P(X/A) : {AttrConcept}\n")
    print(f"P(X) : {probability_list}\n")
    return conceptProbs, AttrConcept, probability_list

In [4]:
def test(examples, Attr, concept_list, conceptProbs, AttrConcept,probability_list):
    misclassification_count = 0
    Total = len(examples)
    for ex in examples:
        px = {}
        for a in Attr:
            for x in ex:
                for c in concept_list:
                    if x in AttrConcept[a]:
                        if c not in px:
                            px[c] = conceptProbs[c] * AttrConcept[a][x][c] /probability_list[a][x]
                        else:
                            px[c] = px[c] * AttrConcept[a][x][c] /probability_list[a][x]
        classification = max(px, key=px.get)
        print(f"Classification : {classification} Expected : {ex[-1]}")
        if (classification != ex[-1]):
            misclassification_count += 1
    misclassification_rate = misclassification_count * 100 / Total
    accuracy = 100 - misclassification_rate
    print(f"Misclassification Count={misclassification_count}")
    print(f"Misclassification Rate={misclassification_rate}%")
    print(f"Accuracy={accuracy}%")


In [5]:
import pandas as pd
df = pd.read_csv('PlayTennies.csv')
concept = str(df.columns[-1])
concept_list = set(df[concept])
Attr = {}
for a in df.columns[:-1]:
    Attr[a] = set(df[a])
    print(f"{a}: {Attr[a]}")
conceptProbs, AttrConcept, probability_list = train(df, Attr, concept_list,concept)
examples = pd.read_csv('PlayTennies.csv')
test(examples.values, Attr, concept_list, conceptProbs, AttrConcept,probability_list)

Outlook: {'Rainy', 'Overcast', 'Sunny'}
Temperature: {'Hot', 'Cool', 'Mild'}
Humidity: {'Normal', 'High'}
Windy: {'Weak', 'Strong'}
P(A) : {'Yes': 0.6428571428571429, 'No': 0.35714285714285715}

P(X/A) : {'Outlook': {'Rainy': {'Yes': 0.3333333333333333, 'No': 0.4}, 'Overcast': {'Yes': 0.4444444444444444, 'No': 0.0}, 'Sunny': {'Yes': 0.2222222222222222, 'No': 0.6}}, 'Temperature': {'Hot': {'Yes': 0.2222222222222222, 'No': 0.4}, 'Cool': {'Yes': 0.3333333333333333, 'No': 0.2}, 'Mild': {'Yes': 0.4444444444444444, 'No': 0.4}}, 'Humidity': {'Normal': {'Yes': 0.6666666666666666, 'No': 0.2}, 'High': {'Yes': 0.3333333333333333, 'No': 0.8}}, 'Windy': {'Weak': {'Yes': 0.6666666666666666, 'No': 0.4}, 'Strong': {'Yes': 0.3333333333333333, 'No': 0.6}}}

P(X) : {'Outlook': {'Rainy': 0.35714285714285715, 'Overcast': 0.2857142857142857, 'Sunny': 0.35714285714285715}, 'Temperature': {'Hot': 0.2857142857142857, 'Cool': 0.2857142857142857, 'Mild': 0.42857142857142855}, 'Humidity': {'Normal': 0.5, 'High': 